In [1]:
import time
start= time.time()
import numpy as np
import pandas as pd
print(time.time()-start)

8.67158031463623


In [2]:
train=pd.read_csv(r"data\train.csv")
test=pd.read_csv(r"data\test.csv")

In [4]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [4]:
test.head()

,tweet_id,tweet
0,7506,Audience Q: What prototyping tools do you use?...
1,7992,At SXSW? Send Your Best Photos &amp; Videos to...
2,247,@mention and here's a pic of you winning your...
3,7688,Google Marissa Mayer: mobile phone as a cursor...
4,3294,#SXSW Google maps is even cooler than I thought


In [3]:
def preprocessing_text(table):
    #put everything in lowercase
    table['tweet'] = table['tweet'].str.lower()
    #Replace rt indicating that was a retweet
    table['tweet'] = table['tweet'].str.replace('rt', '')
    #Replace occurences of mentioning @UserNames
    table['tweet'] = table['tweet'].replace(r'@[^\s]+', 'AT_USER', regex=True)
    #Replace links contained in the tweet
    table['tweet'] = table['tweet'].replace(r'((www\.[^\s]+)|(https?://[^\s]+))', 'URL', regex=True)
    #remove numbers
    table['tweet'] = table['tweet'].replace(r'[0-9]+', '', regex=True)
    #replace special characters and puntuation marks
    table['tweet'] = table['tweet'].replace(r'[!"$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
    table['tweet'] = table['tweet'].replace(r'#([^\s]+)', r'\1', regex=True)
    return table

In [4]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import (
    wordnet,
    stopwords
)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shahzar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shahzar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shahzar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shahzar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
def stop_words(table):
    #We need to remove the stop words
    stop_words_list = stopwords.words('english') + ['sxsw']
    table['tweet'] = table['tweet'].str.lower()
    table['tweet'] = table['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words_list)]))
    return table

In [15]:
def in_dict(word):
    if wordnet.synsets(word):
        #if the word is in the dictionary, we'll return True
        return True

def replace_elongated_word(word):
    regex = r'(\w*)(\w+)\2(\w*)'
    repl = r'\1\2\3'    
    if in_dict(word):
        return word
    new_word = re.sub(regex, repl, word)
    if new_word != word:
        return replace_elongated_word(new_word)
    else:
        return new_word

def detect_elongated_words(row):
    regexrep = r'(\w*)(\w+)(\2)(\w*)'
    words = [''.join(i) for i in re.findall(regexrep, row)]
    for word in words:
        if not in_dict(word):
            row = re.sub(word, replace_elongated_word(word), row)
    return row

In [16]:

def replace_antonyms(word):
    #We get all the lemma for the word
    for syn in wordnet.synsets(word): 
        for lemma in syn.lemmas(): 
            #if the lemma is an antonyms of the word
            if lemma.antonyms(): 
                #we return the antonym
                return lemma.antonyms()[0].name()
    return word

In [17]:
def handling_negation(row):
    #Tokenize the row
    words = word_tokenize(row)
    speach_tags = ['JJ', 'JJR', 'JJS', 'NN', 'VB', 'VBD', 'VBG', 'VBN', 'VBP']
    #We obtain the type of words that we have in the text, we use the pos_tag function
    tags = nltk.pos_tag(words)
    #Now we ask if we found a negation in the words
    tags_2 = ''
    if "n't" in words and "not" in words:
        tags_2 = tags[min(words.index("n't"), words.index("not")):]
        words_2 = words[min(words.index("n't"), words.index("not")):]
        words = words[:(min(words.index("n't"), words.index("not")))+1]
    elif "n't" in words:
        tags_2 = tags[words.index("n't"):]
        words_2 = words[words.index("n't"):] 
        words = words[:words.index("n't")+1]
    elif "not" in words:
        tags_2 = tags[words.index("not"):]
        words_2 = words[words.index("not"):]
        words = words[:words.index("not")+1] 
        
    for index, word_tag in enumerate(tags_2):
        if word_tag[1] in speach_tags:
            words = words+[replace_antonyms(word_tag[0])]+words_2[index+2:]
            break
            
    return ' '.join(words)

In [18]:
def cleaning_table(table):
    #This function will process all the required cleaning for the text in our tweets
    table = preprocessing_text(table)
    table['tweet'] = table['tweet'].apply(lambda x: detect_elongated_words(x))
    table['tweet'] = table['tweet'].apply(lambda x: handling_negation(x))
    table = stop_words(table)
    return table

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7273 entries, 0 to 7273
Data columns (total 3 columns):
tweet_id     7273 non-null int64
tweet        7273 non-null object
sentiment    7273 non-null int64
dtypes: int64(2), object(1)
memory usage: 227.3+ KB


In [20]:
train.dropna(inplace=True)

In [21]:
start= time.time()
train = cleaning_table(train)
print(time.time()- start)

11.03087592124939


In [22]:
test = cleaning_table(test)

In [23]:
train.head()

,tweet_id,tweet,sentiment
0,1701,sxswnui apple defining language touch differen...,1
1,1851,learning ab google doodles doodles light funny...,1
2,2689,one inyourface ex stealing show yrs atuser quo...,2
3,4525,iphone ap would b pretty awesome n't crash ext...,0
4,3604,line outside apple store austin waiting new ip...,1


In [17]:
pd.set_option('display.max_colwidth', -1)
print(train.tweet[:10])

0    sxswnui sxsw apple defining language touch different dialects becoming smaller                                
1    learning ab google doodles doodles light funny amp innovative exceptions significant occasions gogledodle sxsw
2    one inyourface ex stealing show yrs atuser quotat sxsw apple schools mkt expesquot link                       
3    iphone sxsw ap would b pretty awesome n't crash mins extended browsing fuckit ilmakeitwork                    
4    line outside apple store austin waiting new ipad sxsw link                                                    
5    technews one lone dude awaits ipad appleûªs sxsw store link technews apple ipad sxsw tablets tech            
6    sxsw tips prince npr videos toy shopping zuckerberg link sxsw ipad                                            
7    nu user atuser new ubersocial iphone ap store includes uberguide sxsw sponsored mashable                      
8    free sxsw sampler itunes link fremusic                             

In [18]:
pd.set_option('display.max_colwidth', -1)
print(test.tweet[:10])

0    audience q protyping tools use sketchboksharpie pens photoshop balsamic google docs axsure etc myprotype sxsw    
1    sxsw send best photos amp videos link citizenjournalism cn irepo photography sxsw cyber iphone                   
2    atuser 's pic winning ipad unsix sxsw cc atuser atuser link cont link                                            
3    google marisa mayer mobile phone cursor physical location new version map fast real life like sxsw               
4    sxsw google maps even cooler thought                                                                             
5    atuser front atuser popup store sxsw last night link                                                             
6    atuser next life 'm coming back ipad women ca n't discontinue hands thing sxsw                                   
7    google celebrating pi day style sxsw link                                                                        
8    hm bit weird sxsw tending google circle    

In [24]:
from nltk.stem.porter import *
train['tokenized_tweet'] = [nltk.word_tokenize(x) for x in train['tweet']]
stemmer = PorterStemmer()
train['tokenized_tweet'] = train['tokenized_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
train['tokenized_tweet'] = train['tokenized_tweet'].apply(lambda x: ' '.join(x))
train.head()

,tweet_id,tweet,sentiment,tokenized_tweet
0,1701,sxswnui apple defining language touch differen...,1,sxswnui appl defin languag touch differ dialec...
1,1851,learning ab google doodles doodles light funny...,1,learn ab googl doodl doodl light funni amp inn...
2,2689,one inyourface ex stealing show yrs atuser quo...,2,one inyourfac ex steal show yr atus quotat app...
3,4525,iphone ap would b pretty awesome n't crash ext...,0,iphon ap would b pretti awesom n't crash exten...
4,3604,line outside apple store austin waiting new ip...,1,line outsid appl store austin wait new ipad link


In [25]:
test['tokenized_tweet'] = [nltk.word_tokenize(x) for x in test['tweet']]
stemmer = PorterStemmer()
test['tokenized_tweet'] = test['tokenized_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
test['tokenized_tweet'] = test['tokenized_tweet'].apply(lambda x: ' '.join(x))
test.head()

,tweet_id,tweet,tokenized_tweet
0,7506,audience q protyping tools use sketchboksharpi...,audienc q protyp tool use sketchboksharpi pen ...
1,7992,send best photos amp videos link citizenjourna...,send best photo amp video link citizenjourn cn...
2,247,atuser 's pic winning ipad unsix cc atuser atu...,atus 's pic win ipad unsix cc atus atus link c...
3,7688,google marisa mayer mobile phone cursor physic...,googl marisa mayer mobil phone cursor physic l...
4,3294,google maps even cooler thought,googl map even cooler thought


In [26]:
print(train.shape)
print(test.shape)

(7273, 4)
(1819, 3)


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB

In [28]:
X = train['tokenized_tweet']
y = train['sentiment']

In [29]:
cv = CountVectorizer(stop_words=['english',"AT_USER","URL","rt","link","hashtags","amp","gt","quot","sxsw","bit","ly"], ngram_range=(1,2))
X = cv.fit_transform(X)

C:\Users\Shahzar\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['at_user', 'url'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [31]:
# X = tfidf.fit_transform(X)

In [32]:
X.shape

(7273, 38606)

In [33]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=77)

In [28]:
lr = LogisticRegression(random_state=77, class_weight='balanced')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6883593033913841
0.6805236887762856
              precision    recall  f1-score   support

           0       0.52      0.38      0.44       135
           1       0.75      0.80      0.77      1327
           2       0.60      0.56      0.58       693
           3       0.00      0.00      0.00        27

    accuracy                           0.69      2182
   macro avg       0.47      0.43      0.45      2182
weighted avg       0.68      0.69      0.68      2182



In [29]:
from imblearn.under_sampling import TomekLinks
tl=TomekLinks(random_state=42)
X_sample1, y_sample1 = tl.fit_sample(X_train, y_train)
lr.fit(X_sample1, y_sample1)
y_pred1 = lr.predict(X_test)
print(accuracy_score(y_test, y_pred1))
print(f1_score(y_test, y_pred1, average='weighted'))
print(classification_report(y_test, y_pred1))

0.6814848762603116
0.6728002003994007
              precision    recall  f1-score   support

           0       0.53      0.36      0.43       135
           1       0.74      0.81      0.77      1327
           2       0.59      0.53      0.56       693
           3       0.00      0.00      0.00        27

    accuracy                           0.68      2182
   macro avg       0.47      0.43      0.44      2182
weighted avg       0.67      0.68      0.67      2182



In [30]:
# ros = RandomOverSampler(random_state=77)
# log = LogisticRegression(random_state=77)
# X_sample1, y_sample1 = ros.fit_sample(X_train, y_train)
# log.fit(X_sample1, y_sample1)
# y_pred1 = log.predict(X_test)
# print(accuracy_score(y_test, y_pred1))
# print(f1_score(y_test, y_pred1, average='weighted'))
# print(classification_report(y_test, y_pred1))

In [31]:
c2_train=cv.fit_transform(train.tokenized_tweet).toarray()
print(c2_train.shape)
c2_test=cv.transform(test.tokenized_tweet).toarray()
print(c2_test.shape)

(7273, 38813)
(1819, 38813)


In [32]:
y = train['sentiment']

In [33]:

log = LogisticRegression(random_state=77, class_weight='balanced')
#X_sample3, y_sample3 = tl.fit_sample(c2_train, y)
log.fit(c2_train, y)
y_pred3 = log.predict(c2_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [34]:
log = LogisticRegression(random_state=77, class_weight='balanced')
log.fit(c2_train, y)
y_pred3 = log.predict(c2_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [36]:
submission = pd.DataFrame({"tweet_id": test["tweet_id"],"sentiment": y_pred3})
submission.to_csv((r'C:\Users\1403330\Desktop\tweets hackathon\data\sample_submission.csv'), index=False)